In [1]:
from sklearn.metrics import  accuracy_score
from sklearn.datasets import load_iris
from itertools import combinations
import numpy as np
import pandas as pd
import ast

In [2]:
class logisticregression():
    def __init__(self,train_data,train_labels,lr=0.001,batch_size=None,epoch=10,print_every = 10):
        dummy_once = np.ones((len(train_data),1))
        self.train_data = np.hstack((dummy_once,train_data))
        self.train_labels = train_labels

        self.params = np.zeros((len(self.train_data[0]),1))
        
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        self.print_every = print_every
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def cost(self,y,y_pred):
        return -np.mean(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))
    
    def gradient(self,y,y_pred,x):
        return np.dot(x,(y_pred-y))
    
    def train(self):
        for i in range(self.epoch):
            for train,labels in zip(self.train_data,self.train_labels):
                train = train.reshape(-1,1)
                labels = labels.reshape(-1,1)
                y_pred = self.sigmoid(np.dot(train.T,self.params))
                loss = self.cost(labels,y_pred)

                gra = self.gradient(labels,y_pred,train)

                self.params -= self.lr*gra

                if self.print_every:
                    if i%self.print_every == 0 or i == self.epoch-1:
                        print('Epoch : {}  Loss: {}'.format(i,loss))
    def predict(self,test_data):
        result = self.sigmoid(np.dot(test_data,self.params[1:])+self.params[0])
        result[result >= 0.5 ] = 1
        result[result < 0.5 ] = 0
        return result
    
    def evaluate(self,test_data,labels):
        accuracy = accuracy_score(self.predict(test_data),labels)
        return accuracy

In [3]:
def closestNumber(n, m) :
    print('closestNumber: ',n,m)
    # Find the quotient 
    q = int(n / m) 
      
    # 1st possible closest number 
    n1 = m * q 
      
    # 2nd possible closest number 
    if((n * m) > 0) : 
        n2 = (m * (q + 1))  
    else : 
        n2 = (m * (q - 1)) 
      
    # if true, then n1 is the required closest number 
    if (abs(n - n1) < abs(n - n2)) : 
        return n1 
      
    # else n2 is the required closest number  
    return n2

In [4]:
closestNumber(13,4)

closestNumber:  13 4


12

In [15]:
def cross_validate(data,k=5):
    if len(data)%k != 0:
        closest_divisable = closestNumber(len(data),k)
        print('CLOSE',closest_divisable,closest_divisable%k)
        data = data[closest_divisable:,:]
#         return
    
    
    data_splitted = np.vsplit(data,k)
    #TO:DO TRYING TO IMPROVE THE SPLIT
    aggrigate_result = []
    for i in range(len(data_splitted)):
        train = []
        test = []
        items = [j for j in range(len(data_splitted)) if j !=i ]
        test = data_splitted[i]
        for item in items:
            if len(train) == 0:
                train = data_splitted[item]
            else:
                train = np.concatenate((train,data_splitted[item]), axis=0)
            
        logistic = logisticregression(train[:,:-1],train[:,-1:],epoch=100,print_every=100)
        logistic.train()
        
        result = logistic.evaluate(test[:,:-1],test[:,-1:])
        aggrigate_result.append(result)
        
    return aggrigate_result

In [16]:
def get_combinations(items,number):
    comb_list = []
    for c in combinations(items, number):
        c = list(c)
        c.sort()
        comb_list.append(c)
        
    return comb_list

In [17]:
def wrapper_back_method(data,names,target_name,feature_count=2,cross_val_k = 5,visualize=True):
    num = len(names)-1
    classes = []
    if len(names) == feature_count:
        classes = [names]
        return classes
    else:
        classes = get_combinations(names,num)
          
        
    wrapper_output = {}
    for i in classes: 
        cols = np.concatenate((i,target_name),axis=None)
        sub_data = data[cols].values
        result = cross_validate(sub_data,cross_val_k)
        avarage = sum(result)/len(result)

        wrapper_output[str(i)]=avarage
    
    if visualize:       
        print("{} Classess".format(len(classes)),i,'\n',wrapper_output)
    
    wrapper_output_final = wrapper_output
    wrapper_output = sorted(wrapper_output,key=wrapper_output.get)
    
    new_list = [wrapper_output[-1]]
    
    if visualize:
        print('\nSelected -- >\n',new_list,wrapper_output_final[new_list[0]],'\n')
        
    new_list = ast.literal_eval(new_list[0])
    
    if len(new_list)==feature_count:
        return new_list
    else:
        new_list = wrapper_back_method(data,new_list,target_name=target_name,feature_count=feature_count,
                                       cross_val_k=cross_val_k,visualize=visualize)
        return new_list
    

In [18]:
data = load_iris()

data = pd.concat((pd.DataFrame(data['data']),pd.DataFrame(data['target'])),axis=1)
data.columns= ['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)','target']
data = data[(data['target']==0) | (data['target']==1)]
data_np = data.values
np.random.shuffle(data_np)

In [19]:
columns = list(data.columns)
columns.remove('target')


In [20]:
wrapper_back_method(data,names=columns,target_name=['target'],feature_count=3,cross_val_k=10,visualize=True)

Epoch : 0  Loss: 0.6931471805599453
Epoch : 0  Loss: 0.6827270423691651
Epoch : 0  Loss: 0.6742742345238562
Epoch : 0  Loss: 0.6680246626103409
Epoch : 0  Loss: 0.6508832531226673
Epoch : 0  Loss: 0.6375669730127489
Epoch : 0  Loss: 0.6303161572705367
Epoch : 0  Loss: 0.6245813121890459
Epoch : 0  Loss: 0.6076384283497503
Epoch : 0  Loss: 0.6038037405311123
Epoch : 0  Loss: 0.5940468387212795
Epoch : 0  Loss: 0.5869211493195838
Epoch : 0  Loss: 0.5877258775088487
Epoch : 0  Loss: 0.5739742360393667
Epoch : 0  Loss: 0.5687852900795248
Epoch : 0  Loss: 0.5639853821068403
Epoch : 0  Loss: 0.5525536272481454
Epoch : 0  Loss: 0.541317110763452
Epoch : 0  Loss: 0.5356380965900581
Epoch : 0  Loss: 0.539195125765786
Epoch : 0  Loss: 0.5323927957885151
Epoch : 0  Loss: 0.5110380409484511
Epoch : 0  Loss: 0.49846599159005717
Epoch : 0  Loss: 0.4843524304542566
Epoch : 0  Loss: 0.5051652880874818
Epoch : 0  Loss: 0.49756881323468655
Epoch : 0  Loss: 0.47528838833902626
Epoch : 0  Loss: 0.48037998

Epoch : 99  Loss: 0.08355461217130179
Epoch : 99  Loss: 0.13994361952603065
Epoch : 99  Loss: 0.10228234030646918
Epoch : 99  Loss: 0.15638711659626947
Epoch : 99  Loss: 0.07604199256461869
Epoch : 99  Loss: 0.08447466417957382
Epoch : 99  Loss: 0.09942240903731125
Epoch : 99  Loss: 0.10722650681460875
Epoch : 99  Loss: 0.1653622627329775
Epoch : 99  Loss: 0.1443734249813603
Epoch : 99  Loss: 0.13371022602737945
Epoch : 99  Loss: 0.07703739618458047
Epoch : 99  Loss: 0.041833639151206095
Epoch : 99  Loss: 0.15447918224079105
Epoch : 99  Loss: 0.20143064604622116
Epoch : 99  Loss: 0.17692278675809842
Epoch : 99  Loss: 0.12043580420014845
Epoch : 99  Loss: 0.08920543072850358
Epoch : 99  Loss: 0.08356402503560538
Epoch : 99  Loss: 0.15321468320776163
Epoch : 99  Loss: 0.16424785438156836
Epoch : 99  Loss: 0.09268289670149679
Epoch : 99  Loss: 0.04827182603263615
Epoch : 99  Loss: 0.03472239888204403
Epoch : 99  Loss: 0.13632049225698828
Epoch : 99  Loss: 0.07505581470318948
Epoch : 99  L

Epoch : 99  Loss: 0.08502958598571891
Epoch : 99  Loss: 0.1425892353327445
Epoch : 99  Loss: 0.10398828120692
Epoch : 99  Loss: 0.15875300063113343
Epoch : 99  Loss: 0.07725365143834159
Epoch : 99  Loss: 0.08578430909020503
Epoch : 99  Loss: 0.1007968811802054
Epoch : 99  Loss: 0.10898915429861954
Epoch : 99  Loss: 0.16785534551200113
Epoch : 99  Loss: 0.14686495552024195
Epoch : 99  Loss: 0.07414129547742182
Epoch : 99  Loss: 0.13024478262731962
Epoch : 99  Loss: 0.13941710169713206
Epoch : 99  Loss: 0.09552025124465419
Epoch : 99  Loss: 0.030668601750801397
Epoch : 99  Loss: 0.0340818320945995
Epoch : 99  Loss: 0.0431346415150697
Epoch : 99  Loss: 0.0799614326321949
Epoch : 99  Loss: 0.08500884622042393
Epoch : 99  Loss: 0.06929492047616365
Epoch : 99  Loss: 0.13214538257605418
Epoch : 99  Loss: 0.0758911424111185
Epoch : 99  Loss: 0.04127420709193859
Epoch : 99  Loss: 0.15265425817210837
Epoch : 99  Loss: 0.19847466569732805
Epoch : 99  Loss: 0.1753765311846213
Epoch : 99  Loss: 0.1

Epoch : 99  Loss: 0.06761135086006842
Epoch : 99  Loss: 0.11615023089579124
Epoch : 99  Loss: 0.08450397629121179
Epoch : 99  Loss: 0.13057018742964352
Epoch : 99  Loss: 0.061597799847972416
Epoch : 99  Loss: 0.06734187172623428
Epoch : 99  Loss: 0.0819958145797951
Epoch : 99  Loss: 0.08782079201429725
Epoch : 99  Loss: 0.1399392010076636
Epoch : 99  Loss: 0.12012198416875068
Epoch : 99  Loss: 0.05857559257097779
Epoch : 99  Loss: 0.1061369736461346
Epoch : 99  Loss: 0.11470962680436064
Epoch : 99  Loss: 0.07930352713764885
Epoch : 99  Loss: 0.02368951512263416
Epoch : 99  Loss: 0.02622124778068376
Epoch : 99  Loss: 0.033881992289285734
Epoch : 99  Loss: 0.06426677401384154
Epoch : 99  Loss: 0.06691115147717575
Epoch : 99  Loss: 0.05535389564861137
Epoch : 99  Loss: 0.10651562781720955
Epoch : 99  Loss: 0.06089838341034927
Epoch : 99  Loss: 0.033460414746917085
Epoch : 99  Loss: 0.1247996329452133
Epoch : 99  Loss: 0.16421836909803258
Epoch : 99  Loss: 0.14520535531577783
Epoch : 99  L

Epoch : 99  Loss: 0.06755503456467885
Epoch : 99  Loss: 0.1157333724672807
Epoch : 99  Loss: 0.08444088181384583
Epoch : 99  Loss: 0.1307236005984729
Epoch : 99  Loss: 0.06166727238344382
Epoch : 99  Loss: 0.06754426627078239
Epoch : 99  Loss: 0.08224964740115273
Epoch : 99  Loss: 0.08783394495496422
Epoch : 99  Loss: 0.13997356871362773
Epoch : 99  Loss: 0.11991653065368663
Epoch : 99  Loss: 0.05852377973278158
Epoch : 99  Loss: 0.10645957123107276
Epoch : 99  Loss: 0.11440608144211564
Epoch : 99  Loss: 0.07924743146437495
Epoch : 99  Loss: 0.02357282276390941
Epoch : 99  Loss: 0.026299582063113586
Epoch : 99  Loss: 0.03385116580716571
Epoch : 99  Loss: 0.06421345071672259
Epoch : 99  Loss: 0.06685025672685324
Epoch : 99  Loss: 0.055523226886376446
Epoch : 99  Loss: 0.10632546424514898
Epoch : 99  Loss: 0.0610252240254635
Epoch : 99  Loss: 0.033499663386756647
Epoch : 99  Loss: 0.12481913460797567
Epoch : 99  Loss: 0.1651622450324745
Epoch : 99  Loss: 0.14482168646873203
Epoch : 99  L

...

Epoch : 99  Loss: 0.07975591714852197
Epoch : 99  Loss: 0.142203061242673
Epoch : 99  Loss: 0.09913667850071439
Epoch : 99  Loss: 0.143728447930407
Epoch : 99  Loss: 0.07046754783441321
Epoch : 99  Loss: 0.08245778102002446
Epoch : 99  Loss: 0.09429245470344327
Epoch : 99  Loss: 0.1012002632442604
Epoch : 99  Loss: 0.15800024882902866
Epoch : 99  Loss: 0.1347328918582385
Epoch : 99  Loss: 0.07079333076381582
Epoch : 99  Loss: 0.11450227135777152
Epoch : 99  Loss: 0.13249210970218311
Epoch : 99  Loss: 0.08980367310063773
Epoch : 99  Loss: 0.03297696734467869
Epoch : 99  Loss: 0.034286455861829074
Epoch : 99  Loss: 0.04784674680145357
Epoch : 99  Loss: 0.0827536634597289
Epoch : 99  Loss: 0.08595220707005193
Epoch : 99  Loss: 0.0660034214121124
Epoch : 99  Loss: 0.12896744145053554
Epoch : 99  Loss: 0.07857097355346354
Epoch : 99  Loss: 0.04050777032101718
Epoch : 99  Loss: 0.1707405881388345
Epoch : 99  Loss: 0.1652435349850299
Epoch : 99  Loss: 0.17798762813462574
Epoch : 99  Loss: 0.1

Epoch : 99  Loss: 0.06333038294173818
Epoch : 99  Loss: 0.11691250447869225
Epoch : 99  Loss: 0.08037123291550645
Epoch : 99  Loss: 0.11745235916985877
Epoch : 99  Loss: 0.055637536143505105
Epoch : 99  Loss: 0.0639284400216358
Epoch : 99  Loss: 0.0755600924445011
Epoch : 99  Loss: 0.08105772539819082
Epoch : 99  Loss: 0.13101387779563411
Epoch : 99  Loss: 0.11013287619855266
Epoch : 99  Loss: 0.055609771051887726
Epoch : 99  Loss: 0.09183652235458971
Epoch : 99  Loss: 0.10907667555251445
Epoch : 99  Loss: 0.0738959546055899
Epoch : 99  Loss: 0.02545688405044836
Epoch : 99  Loss: 0.025804890389256516
Epoch : 99  Loss: 0.03721398008094679
Epoch : 99  Loss: 0.06602358296629841
Epoch : 99  Loss: 0.06724790735510859
Epoch : 99  Loss: 0.051648812958244986
Epoch : 99  Loss: 0.10367610822701728
Epoch : 99  Loss: 0.061977986507845556
Epoch : 99  Loss: 0.03215518930082551
Epoch : 99  Loss: 0.13880731644062405
Epoch : 99  Loss: 0.13318780708627373
Epoch : 99  Loss: 0.14727170711523171
Epoch : 99

Epoch : 99  Loss: 0.06361971097173046
Epoch : 99  Loss: 0.11744519767167678
Epoch : 99  Loss: 0.08071460604100814
Epoch : 99  Loss: 0.11802009632371407
Epoch : 99  Loss: 0.05589019711563535
Epoch : 99  Loss: 0.06433351038688466
Epoch : 99  Loss: 0.07593756948006092
Epoch : 99  Loss: 0.08144934551117121
Epoch : 99  Loss: 0.13160545469856944
Epoch : 99  Loss: 0.11061371912412081
Epoch : 99  Loss: 0.05587686344613326
Epoch : 99  Loss: 0.09230473303575364
Epoch : 99  Loss: 0.10952024505317286
Epoch : 99  Loss: 0.07413131119396246
Epoch : 99  Loss: 0.025553863830281787
Epoch : 99  Loss: 0.02595013063858552
Epoch : 99  Loss: 0.03740274716147398
Epoch : 99  Loss: 0.06635025617740556
Epoch : 99  Loss: 0.06763922937075564
Epoch : 99  Loss: 0.05191830637549173
Epoch : 99  Loss: 0.10423027699913057
Epoch : 99  Loss: 0.062329383599756365
Epoch : 99  Loss: 0.032258607100952456
Epoch : 99  Loss: 0.1397249443837206
Epoch : 99  Loss: 0.13396789315924382
Epoch : 99  Loss: 0.1480066803653466
Epoch : 99 

Epoch : 99  Loss: 0.061683655467477325
Epoch : 99  Loss: 0.11397226498175916
Epoch : 99  Loss: 0.07838910002967263
Epoch : 99  Loss: 0.1143894472489887
Epoch : 99  Loss: 0.05419010157825773
Epoch : 99  Loss: 0.06207899485318569
Epoch : 99  Loss: 0.07362100359860296
Epoch : 99  Loss: 0.07890728588172712
Epoch : 99  Loss: 0.12776704857695725
Epoch : 99  Loss: 0.10726883926342798
Epoch : 99  Loss: 0.05411678329735935
Epoch : 99  Loss: 0.08933511074188254
Epoch : 99  Loss: 0.10635432794357204
Epoch : 99  Loss: 0.07223755349008361
Epoch : 99  Loss: 0.02483994689017097
Epoch : 99  Loss: 0.02509989191896644
Epoch : 99  Loss: 0.03628176522656229
Epoch : 99  Loss: 0.06433669039719979
Epoch : 99  Loss: 0.06533135365370542
Epoch : 99  Loss: 0.05026828071788676
Epoch : 99  Loss: 0.10078860317776014
Epoch : 99  Loss: 0.06033444124462935
Epoch : 99  Loss: 0.03145116273176689
Epoch : 99  Loss: 0.13501500561937746
Epoch : 99  Loss: 0.12929574962172088
Epoch : 99  Loss: 0.1434272908607453
Epoch : 99  L

['petal length (cm)', 'petal width (cm)', 'sepal width (cm)']

In [21]:
data = pd.read_csv('../../dataset_playground/data/Electricity-problem/electricity-normalized.csv')

In [22]:
data['target_class']=data['class'].apply(lambda x: 1 if x.lower()=='up' else 0)
col_names = data.columns
col_names

Index(['date', 'day', 'period', 'nswprice', 'nswdemand', 'vicprice',
       'vicdemand', 'transfer', 'class', 'target_class'],
      dtype='object')

In [23]:
elec_data = data[['date', 'day', 'period', 'nswprice', 'nswdemand', 'vicprice',
       'vicdemand', 'transfer', 'target_class']]

In [ ]:
wrapper_back_method(elec_data,['date', 'day', 'period', 'nswprice', 'nswdemand', 'vicprice',
       'vicdemand', 'transfer'],[ 'target_class'],feature_count=7,cross_val_k=2,visualize=True)

Epoch : 0  Loss: 0.6931471805599453
Epoch : 0  Loss: 0.6998510723394604
Epoch : 0  Loss: 0.6931073268758003
Epoch : 0  Loss: 0.6864564130391808
Epoch : 0  Loss: 0.6799264156665896
Epoch : 0  Loss: 0.6735125499689775
Epoch : 0  Loss: 0.6672096840931404
Epoch : 0  Loss: 0.6609972841299284
Epoch : 0  Loss: 0.6548607441725949
Epoch : 0  Loss: 0.6487955802123929
Epoch : 0  Loss: 0.6427713089294536
Epoch : 0  Loss: 0.6367875661017683
Epoch : 0  Loss: 0.6307715541673221
Epoch : 0  Loss: 0.6247821193062694
Epoch : 0  Loss: 0.618944804132633
Epoch : 0  Loss: 0.7801900370717125
Epoch : 0  Loss: 0.7723508119643375
Epoch : 0  Loss: 0.7645009483426434
Epoch : 0  Loss: 0.7567972545383991
Epoch : 0  Loss: 0.7491605756146242
Epoch : 0  Loss: 0.7416114009517065
Epoch : 0  Loss: 0.6537270678163827
Epoch : 0  Loss: 0.647520081407204
Epoch : 0  Loss: 0.6413980813873764
Epoch : 0  Loss: 0.635366769750145
Epoch : 0  Loss: 0.6294064635568889
Epoch : 0  Loss: 0.6235015345928397
Epoch : 0  Loss: 0.617689254888

Epoch : 0  Loss: 0.6947248151084153
Epoch : 0  Loss: 0.6914825392007489
Epoch : 0  Loss: 0.6911498768910987
Epoch : 0  Loss: 0.610747110686735
Epoch : 0  Loss: 0.608729181870465
Epoch : 0  Loss: 0.7850850510188141
Epoch : 0  Loss: 0.7841284921742626
Epoch : 0  Loss: 0.7838824067804409
Epoch : 0  Loss: 0.610305365530462
Epoch : 0  Loss: 0.6080753329009702
Epoch : 0  Loss: 0.6067183230011675
Epoch : 0  Loss: 0.6056735257422258
Epoch : 0  Loss: 0.6050337267987129
Epoch : 0  Loss: 0.6049347930983848
Epoch : 0  Loss: 0.6053345736295359
Epoch : 0  Loss: 0.6073240755438288
Epoch : 0  Loss: 0.7841342760055072
Epoch : 0  Loss: 0.6128450390163268
Epoch : 0  Loss: 0.7777584447418675
Epoch : 0  Loss: 0.7726287990415998
Epoch : 0  Loss: 0.7686268619724113
Epoch : 0  Loss: 0.7639418560943545
Epoch : 0  Loss: 0.7609351176531596
Epoch : 0  Loss: 0.7584081288533646
Epoch : 0  Loss: 0.7566577264123351
Epoch : 0  Loss: 0.7548344032499714
Epoch : 0  Loss: 0.7526111990473562
Epoch : 0  Loss: 0.751118401248

Epoch : 0  Loss: 1.0408455800515761
Epoch : 0  Loss: 1.0327101401835774
Epoch : 0  Loss: 1.0236505162360132
Epoch : 0  Loss: 1.0143525474457842
Epoch : 0  Loss: 1.0052649179131201
Epoch : 0  Loss: 0.9977935668457438
Epoch : 0  Loss: 0.9904861900656309
Epoch : 0  Loss: 0.46630782713019026
Epoch : 0  Loss: 0.4629575100887981
Epoch : 0  Loss: 0.4592668835214757
Epoch : 0  Loss: 0.4575324949543166
Epoch : 0  Loss: 0.4570137117842494
Epoch : 0  Loss: 0.4549438556665187
Epoch : 0  Loss: 0.4513782418557409
Epoch : 0  Loss: 0.44707126433401695
Epoch : 0  Loss: 0.4457227646073435
Epoch : 0  Loss: 0.4435692168995958
Epoch : 0  Loss: 0.4419510854978167
Epoch : 0  Loss: 0.44053548797829883
Epoch : 0  Loss: 1.0285846641722123
Epoch : 0  Loss: 0.4466579213164365
Epoch : 0  Loss: 0.34818861796360573
Epoch : 0  Loss: 0.34409036851223507
Epoch : 0  Loss: 0.341823999524006
Epoch : 0  Loss: 0.3378751812439553
Epoch : 0  Loss: 0.33390249583952913
Epoch : 0  Loss: 0.33046672868405536
Epoch : 0  Loss: 0.327

Epoch : 0  Loss: 0.5777464668437301
Epoch : 0  Loss: 0.5728269659234692
Epoch : 0  Loss: 0.5702008468625369
Epoch : 0  Loss: 0.5684894088606421
Epoch : 0  Loss: 0.5647905491346257
Epoch : 0  Loss: 0.5582741778205343
Epoch : 0  Loss: 0.8528859790559545
Epoch : 0  Loss: 0.5584176144295403
Epoch : 0  Loss: 0.5556989542864995
Epoch : 0  Loss: 0.5522231565437058
Epoch : 0  Loss: 0.552838228094708
Epoch : 0  Loss: 0.5479284305265013
Epoch : 0  Loss: 0.4520084260220018
Epoch : 0  Loss: 0.4444221161315991
Epoch : 0  Loss: 0.4413743417399911
Epoch : 0  Loss: 0.43556994264828497
Epoch : 0  Loss: 0.4291599542206066
Epoch : 0  Loss: 0.4241907016122926
Epoch : 0  Loss: 0.4206947720606799
Epoch : 0  Loss: 0.4184202211829265
Epoch : 0  Loss: 0.41796612459677485
Epoch : 0  Loss: 0.41757791852190673
Epoch : 0  Loss: 1.066903481326614
Epoch : 0  Loss: 1.0411502822466412
Epoch : 0  Loss: 1.018428777880904
Epoch : 0  Loss: 0.9936976269843336
Epoch : 0  Loss: 0.9807954291383592
Epoch : 0  Loss: 0.479206945

Epoch : 0  Loss: 0.8216939436587395
Epoch : 0  Loss: 0.8063468322969842
Epoch : 0  Loss: 0.8047935314080147
Epoch : 0  Loss: 0.7967698287245908
Epoch : 0  Loss: 0.7869646185733417
Epoch : 0  Loss: 0.7763319127702852
Epoch : 0  Loss: 0.7707703299701116
Epoch : 0  Loss: 0.7667019361828206
Epoch : 0  Loss: 0.764288400825789
Epoch : 0  Loss: 0.6244742725331939
Epoch : 0  Loss: 0.6149640153306557
Epoch : 0  Loss: 0.6058038662589695
Epoch : 0  Loss: 0.5996259992733826
Epoch : 0  Loss: 0.5953478271964534
Epoch : 0  Loss: 0.5913973090888307
Epoch : 0  Loss: 0.5842927209739316
Epoch : 0  Loss: 0.5762440080762985
Epoch : 0  Loss: 0.5716546826244033
Epoch : 0  Loss: 0.5654268913075858
Epoch : 0  Loss: 0.5613708703452462
Epoch : 0  Loss: 0.5566422762812985
Epoch : 0  Loss: 0.5580313833523087
Epoch : 0  Loss: 0.5523089899888081
Epoch : 0  Loss: 0.45504081794344586
Epoch : 0  Loss: 0.44710580494529384
Epoch : 0  Loss: 0.44408960068332
Epoch : 0  Loss: 0.4379935462662307
Epoch : 0  Loss: 0.4308381562

...

Epoch : 0  Loss: 0.5541772880685526
Epoch : 0  Loss: 0.5537869508272054
Epoch : 0  Loss: 0.5584634011404128
Epoch : 0  Loss: 0.5612959652528456
Epoch : 0  Loss: 0.5629833878558398
Epoch : 0  Loss: 0.5704985733399949
Epoch : 0  Loss: 0.5798618623839705
Epoch : 0  Loss: 0.5880110163618096
Epoch : 0  Loss: 0.7993726985256958
Epoch : 0  Loss: 0.60301166479199
Epoch : 0  Loss: 0.7871872992315505
Epoch : 0  Loss: 0.7835048832352904
Epoch : 0  Loss: 0.7775326477408189
Epoch : 0  Loss: 0.7709515636570324
Epoch : 0  Loss: 0.7743281409296805
Epoch : 0  Loss: 0.7759820030526211
Epoch : 0  Loss: 0.7790290589928563
Epoch : 0  Loss: 0.7805127646164696
Epoch : 0  Loss: 0.6138195868050392
Epoch : 0  Loss: 0.6110389342814686
Epoch : 0  Loss: 0.6116869952282773
Epoch : 0  Loss: 0.6147203490233827
Epoch : 0  Loss: 0.6184127106399971
Epoch : 0  Loss: 0.627082471482869
Epoch : 0  Loss: 0.7484939473111103
Epoch : 0  Loss: 0.7273303797195484
Epoch : 0  Loss: 0.7244023130723758
Epoch : 0  Loss: 0.727382886871

Epoch : 0  Loss: 0.8584177266462624
Epoch : 0  Loss: 0.8557496767794658
Epoch : 0  Loss: 0.8561941408689744
Epoch : 0  Loss: 0.8509639671456459
Epoch : 0  Loss: 0.5540588154292886
Epoch : 0  Loss: 0.48877507854038915
Epoch : 0  Loss: 0.4849318789678738
Epoch : 0  Loss: 0.9571331544856944
Epoch : 0  Loss: 0.48309836108760484
Epoch : 0  Loss: 0.477726710199332
Epoch : 0  Loss: 0.4726185953686198
Epoch : 0  Loss: 0.46782794446257514
Epoch : 0  Loss: 0.46503727943201373
Epoch : 0  Loss: 0.46337857983860503
Epoch : 0  Loss: 0.4634911680132302
Epoch : 0  Loss: 0.4672417567163295
Epoch : 0  Loss: 0.47199823837637256
Epoch : 0  Loss: 0.48361882310395193
Epoch : 0  Loss: 0.4943128401793377
Epoch : 0  Loss: 0.4991687647664783
Epoch : 0  Loss: 0.9167923235384797
Epoch : 0  Loss: 0.5183626185261089
Epoch : 0  Loss: 0.5226311247195852
Epoch : 0  Loss: 0.8935024357632966
Epoch : 0  Loss: 0.5283473787678888
Epoch : 0  Loss: 0.526301126522375
Epoch : 0  Loss: 0.8957137184422822
Epoch : 0  Loss: 0.8952

Epoch : 0  Loss: 0.7231383353206234
Epoch : 0  Loss: 0.7195382089830965
Epoch : 0  Loss: 0.7177602788918716
Epoch : 0  Loss: 0.7170795417426182
Epoch : 0  Loss: 0.719430621474339
Epoch : 0  Loss: 0.7263117095822029
Epoch : 0  Loss: 0.7399023171481883
Epoch : 0  Loss: 0.6232804058827437
Epoch : 0  Loss: 0.6095876549584973
Epoch : 0  Loss: 0.6093838266419968
Epoch : 0  Loss: 0.6153293927236352
Epoch : 0  Loss: 0.6192771378568038
Epoch : 0  Loss: 0.6219268076155973
Epoch : 0  Loss: 0.6246737823536557
Epoch : 0  Loss: 0.6300760046163978
Epoch : 0  Loss: 0.7497440241269915
Epoch : 0  Loss: 0.6390734213795716
Epoch : 0  Loss: 0.6428107270601024
Epoch : 0  Loss: 0.629743421186522
Epoch : 0  Loss: 0.6301973823252346
Epoch : 0  Loss: 0.6835874877991956
Epoch : 0  Loss: 0.6860696943397072
Epoch : 0  Loss: 0.6792021803642414
Epoch : 0  Loss: 0.6856442000362502
Epoch : 0  Loss: 0.6919777539053736
Epoch : 0  Loss: 0.6789608659097165
Epoch : 0  Loss: 0.6644843494382379
Epoch : 0  Loss: 0.65671094457

Epoch : 0  Loss: 0.6295547679505246
Epoch : 0  Loss: 0.6265758339112806
Epoch : 0  Loss: 0.6232972279660978
Epoch : 0  Loss: 0.6208346623223753
Epoch : 0  Loss: 0.6153459957609675
Epoch : 0  Loss: 0.6120222496866087
Epoch : 0  Loss: 0.6086576502702775
Epoch : 0  Loss: 0.6079533231482851
Epoch : 0  Loss: 0.6054263063206005
Epoch : 0  Loss: 0.6004548273249294
Epoch : 0  Loss: 0.595787182430572
Epoch : 0  Loss: 0.5888768853841769
Epoch : 0  Loss: 0.5823900339944091
Epoch : 0  Loss: 0.567178608795466
Epoch : 0  Loss: 0.5556885202269682
Epoch : 0  Loss: 0.5552076783701717
Epoch : 0  Loss: 0.8414936922807237
Epoch : 0  Loss: 0.8249358602369679
Epoch : 0  Loss: 0.814517371228147
Epoch : 0  Loss: 0.8054418322534986
Epoch : 0  Loss: 0.79052381345969
Epoch : 0  Loss: 0.7761926999121699
Epoch : 0  Loss: 0.6069296621665712
Epoch : 0  Loss: 0.7917958520387238
Epoch : 0  Loss: 0.5869989582391727
Epoch : 0  Loss: 0.8141727532745021
Epoch : 0  Loss: 0.6627177900885919
Epoch : 0  Loss: 0.72135107319086

Epoch : 0  Loss: 0.8901512026809338
Epoch : 0  Loss: 0.87726527359172
Epoch : 0  Loss: 0.870378440395752
Epoch : 0  Loss: 0.8623338575173538
Epoch : 0  Loss: 0.8503392631375837
Epoch : 0  Loss: 0.8416726952197779
Epoch : 0  Loss: 0.8361698667116971
Epoch : 0  Loss: 0.8341504153250734
Epoch : 0  Loss: 0.8246932606792493
Epoch : 0  Loss: 0.8151601584632048
Epoch : 0  Loss: 0.8050715037972871
Epoch : 0  Loss: 0.7947991074008711
Epoch : 0  Loss: 0.7881904121041297
Epoch : 0  Loss: 0.7836343324633369
Epoch : 0  Loss: 0.6013627296580571
Epoch : 0  Loss: 0.595477504308392
Epoch : 0  Loss: 0.5875456084127868
Epoch : 0  Loss: 0.5863944954415331
Epoch : 0  Loss: 0.5849481053803983
Epoch : 0  Loss: 0.5769391414150844
Epoch : 0  Loss: 0.5646811053055157
Epoch : 0  Loss: 0.5567834518226967
Epoch : 0  Loss: 0.5522361040062334
Epoch : 0  Loss: 0.5456035445370848
Epoch : 0  Loss: 0.5468456000681755
Epoch : 0  Loss: 0.5457763938638108
Epoch : 0  Loss: 0.8402524703882488
Epoch : 0  Loss: 0.5658215654618